In [1]:
"""Checking a portfolio for transaction updates since an AsAt time


Attributes
----------
AsAt
Transaction Portfolios
GetPortfolioChanges API
"""

'Checking a portfolio for transaction updates since an AsAt time\n\n\nAttributes\n----------\nAsAt\nTransaction Portfolios\nGetPortfolioChanges API\n'

# 1. Introduction

In this notebook, we show how you can call LUSID to check if there have been any new, amended, or canceled transactions since a given `AsAt` datetime. From a technical point-of-view this is a two step process:

1. Call [GetPortfolioChanges](https://www.lusid.com/docs/api/#operation/GetPortfolioChanges) endpoint for a given `scope` and `AsAt` date. From that response, we get a list of all portfolios in that scope which have changes after the AsAt date.
2. Call [GetTransactions](https://www.lusid.com/docs/api/#operation/GetTransactions) on the list of portfolios from step #1, filtering for Transactions with a TransactionEntryDate after the asAt Date

> NOTE: We implement a two step check here with <b>GetPortfolioChanges</b> as we don't want to repeatedly call <b>GetTransactions</b> for every portfolio in a scope if there are no changes to be fetched.

![GetPortfolioChanges1](img/get-portfolio-changes/get-portfolio-changes.gif)

# 2. Setup LUSID

In [2]:
# Import general purpose packages
import os
import json
from datetime import datetime, timedelta, date
import pytz
from time import sleep

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import data wrangling packages
import pandas as pd

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [3]:
# Define the transaction portfolio API

txn_port_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
entities_api = api_factory.build(lusid.api.EntitiesApi)
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)

In [4]:
# Load a mapping file for DataFrame headers for the get holdings response
with open(r"config/get_holdings_mapping.json") as mappings_file:
    get_holdings_json_mapping = json.load(mappings_file)

# Load a mapping file for DataFrame headers for the get transactions response
with open(r"config/get_transactions_mapping.json") as mappings_file:
    get_txn_json_mapping = json.load(mappings_file)

In [5]:
def datetime_to_filter_string(entry_datetime):
    
    date_obj = entry_datetime
    
    return date_obj.isoformat()[:-6] + "0Z"

# 3. Create two sample portfolio for demonstration

Here we create two portfolios for demonstration, each containing UK equities from the FTSE100. We upload the transactions in two batches, one for each portfolio. We record the AsAt time from the second batch, this is the time the batch is considered "completed".

In [6]:
# Create a new scope

scope = create_scope_id()
portfolio_code1 = "EQUITY_UK1"
portfolio_code2 = "EQUITY_UK2"

In [7]:
# Load a file of equity transactions

transactions_file = r"data/get-portfolio-changes/equity_transactions.csv"
transactions_df = pd.read_csv(transactions_file)

In [8]:
transactions_df.head(2)

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions
0,EQUITY_UK1,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0001,Buy,02/01/2020,04/01/2020,120000,5,600000,GBP,ftse_tracker,NaN
1,EQUITY_UK1,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0002,Buy,02/01/2020,04/01/2020,12000,5,60000,GBP,ftse_tracker,NaN


In [ ]:
# Load portfolios, instruments, and transactions

seed_data_response = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_file,
    "csv",
)

We capture the latest AsAt from the batch of upsert transaction responses. The transactions for each portfolio will have been upserted to LUSID using separate requests, so will have slightly different AsAt timestamps. As such we need to find the latest AsAt and use this as time of the batch completion. 

In [ ]:
equity_batch_last_asat_response = max(
    [i.version.as_at_date for i in seed_data_response["transactions"][0]["transactions"]["success"]])

In [ ]:
equity_batch_last_asat = datetime_to_filter_string(equity_batch_last_asat_response)

print(f"AsAt of last transaction in the transaction upsert batch {equity_batch_last_asat}")

We sleep for 2 seconds to create a short break in the asat timeline, so we can compare results below.

In [ ]:
sleep(2)

# 4. Book a batch of fund flows for both portfolios

Next, we book a batch of fund flows into both funds. The first set of transactions are all backdated. We book some current and future dated transactions later.

In [ ]:
# Define now in UTC
now = datetime.now(tz=pytz.UTC)

# Get SOD (midnight) today
today = date.today()
today_sod = pytz.utc.localize(datetime.combine(today, datetime.min.time()))

# Create upsert datetimes for yesterday
eight_pm_yesterday = today_sod - timedelta(hours=4)
ten_pm_yesterday = today_sod - timedelta(hours=2)

# Create upsert datetimes for today
nine_am_today = today_sod + timedelta(hours=9)
eleven_am_today = today_sod + timedelta(hours=11)

# Create upsert datetimes for tomorrow
nine_am_tomorrow = today_sod + timedelta(hours=33)

Define function to manage fund flow upsert:

In [ ]:
def upsert_fund_flow_batch(fund_flow_txns):
    
    """
    This function upserts a set of fund flows into LUSID. 
    We purposefully leave 1 second between each upsert. 
    The function returns a the AsAt of last upsert
    
    """
    
    upsert_asat_times = []
    
    for portfolio_code in fund_flow_txns:
        
        sleep(1)
        
        transactions = fund_flow_txns[portfolio_code]
        
        upsert_transactions = txn_port_api.upsert_transactions(
            scope=scope,
            code=portfolio_code,
            transaction_request=[
                models.TransactionRequest(
                    transaction_id=transaction[0],
                    type=transaction[1],
                    instrument_identifiers={"Instrument/default/LusidInstrumentId": f"CCY_{transaction[2]}"},
                    transaction_date=transaction[3],
                    settlement_date=transaction[3],
                    units=transaction[4],
                    transaction_price=models.TransactionPrice(price=1, type="Price"),
                    total_consideration=models.CurrencyAndAmount(amount=transaction[4], currency=transaction[2]),
                    properties={},
                )
             for transaction in transactions]
        )
    
        upsert_asat_times.append(upsert_transactions.version.as_at_date)
                            
    return upsert_asat_times

In [ ]:
fund_flow_txns_batch_1 = {
    
    # Post transactions for portfolio 1

    portfolio_code1: [ ("funds_001", "FundsIn", "GBP", eight_pm_yesterday, 100000),
    ("funds_002", "FundsIn", "USD", ten_pm_yesterday, 300000)],
    
    # Post transactions for portfolio 2

    portfolio_code2: [
            ("funds_001", "FundsIn", "GBP", eight_pm_yesterday, 100000),
    ("funds_002", "FundsIn", "USD", ten_pm_yesterday, 300000)
            
    ]
        
}

Upsert the fund flow transactions:

In [ ]:
fund_flow_batch_1_upsert_asats = upsert_fund_flow_batch(fund_flow_txns_batch_1)

Again, we capture the AsAt of the last API request in the batch. This AsAt is used to filter transactions below.

In [ ]:
fund_flow_batch_1_end_time = datetime_to_filter_string(max(fund_flow_batch_1_upsert_asats))

print(f"The AsAt of the last API request in the batch: {fund_flow_batch_1_end_time}")

# 5. Track the transaction updates since fund flow posting


In this section we retrieve all transactions which were posted after the first batch of equity transactions. We use the AsAt time to filter. 


<i>Diagram - Querying all transactions by EntryDateTime greater than the first EntryDateTime in the current batch:</i>


![GetPortfolioChanges2](img/get-portfolio-changes/get-changes-1.gif)


In [ ]:
def get_latest_changes_entered_since_asat_date(scope, last_checked_date, operator="gt"):
    
    """
    This function returns all the transactions posted into all portfolios in a scope since 
    an AsAt time provided by the user.
    
    There is two step process:
    1. Call the GetPortfolioChanges API to get the list of portfolios with changes
    2. Call the GetTransactions API to get details of transactions for a given portfolio
    
    """
    
    # The max_date resolves to the year 9999
    # We use this as a proxy to check for any changes across all effective space
    
    max_date = datetime.max.isoformat() + "Z"
    
    
    def get_transactions_since_date(portfolio_code, last_checked_date):        
        
        api_filter = f"entryDateTime {operator} {last_checked_date}"
                                
        get_transactions = txn_port_api.get_transactions(
            scope=scope,
            code=portfolio_code,
            filter=api_filter)
        
        txn_df = lusid_response_to_data_frame(get_transactions, rename_properties=True, column_name_mapping=get_txn_json_mapping )
        
        txn_df=txn_df[get_txn_json_mapping.values()]
        
        return txn_df
    
    portfolios_with_changes =  [i.entity_id.code for i in entities_api.get_portfolio_changes(
        scope=scope, effective_at=max_date, as_at=last_checked_date).values]
    
    
    for portfolio in portfolios_with_changes:
        
        print(f"Fetching transactions for portfolio {portfolio} since AsAt time {last_checked_date}")
        
        txns_df = get_transactions_since_date(portfolio, last_checked_date)
        
        display(txns_df)
        
    

Track the changes in the portfolio since the first upsert:

In [ ]:
get_latest_changes_entered_since_asat_date(scope, equity_batch_last_asat)

# 6. Post new and corrections to fund flow transactions

Then we post new transactions and a corrections. We add a correction here to illustrate that transactions are filtered by AsAt and not EffectiveAt. We also upsert transactions into one portfolio only, to demonstrate that [GetPortfolioChanges](https://www.lusid.com/docs/api/#operation/GetPortfolioChanges) only returns portfolios which actually have had changes.

In [ ]:
fund_flow_txns_batch_2 = {
    
    # Post transactions for portfolio 2
    
    portfolio_code2: [
        
        ("funds_001", "FundsIn", "GBP", eight_pm_yesterday, 150000),
        ("funds_003", "FundsIn", "EUR", nine_am_today, 200000),
        ("funds_004", "FundsOut", "GBP", eleven_am_today, 200000),
        ("funds_005", "FundsIn", "GBP", nine_am_tomorrow, 200000)
    ]
    
}

In [ ]:
fund_flow_upsert_2 = upsert_fund_flow_batch(fund_flow_txns_batch_2)

# 7. Get the latest transactions cut

Next we take another transactions cut for anything booked after the first batch of fund flows.

<i>Diagram - Querying all transactions by EntryDateTime greater than the last EntryDateTime in the previous batch:</i>

![GetPortfolioChanges3](img/get-portfolio-changes/get-changes-2.gif)


We can see below that the latest cut only includes:

1. New transactions
2. Corrections to old transactions posted since previous call

In [ ]:
get_latest_changes_entered_since_asat_date(scope, fund_flow_batch_1_end_time, operator="gt")

# 8. Clean-up: Delete portfolios

In [ ]:
for port in (portfolio_code1, portfolio_code2):

    portfolios_api.delete_portfolio(scope=scope, code=port)